In [1]:
# load appropriate modules
import numpy as np
import xarray as xr
from xmitgcm import open_mdsdataset
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from xgcm import Grid

In [2]:
import dask
from multiprocessing.pool import ThreadPool
dask.set_options(pool=ThreadPool(16))

In [3]:
# load data 
nsteps = range(4665600, 4665600+103680, (4665744-4665600))
data_dir = '/swot/SUM05/dbalwada/channel_model_output/varying_res/02km/run_5km_start_tracer/'
#ds = open_mdsdataset(data_dir, delta_t=300, prefix=['U','V','T','W','Eta']
#                     ,ignore_unknown_vars=True, geometry='cartesian')
ds_tracer_2 = open_mdsdataset(data_dir, delta_t=300, iters= nsteps, prefix=['PTRACER01','T','W','V','U']
                     ,ignore_unknown_vars=True, geometry='cartesian', ref_date = "2100-1-1 0:0:0")

/home/dbalwada/miniconda/envs/dhruvenv/lib/python2.7/site-packages/xmitgcm/mds_store.py:696: UserWarning: Couldn't find available_diagnostics.log in /swot/SUM05/dbalwada/channel_model_output/varying_res/02km/run_5km_start_tracer/. Using default version.
  "in %s. Using default version." % data_dir)
/home/dbalwada/miniconda/envs/dhruvenv/lib/python2.7/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)


In [4]:
dir_mean= '/swot/SUM05/dbalwada/channel_model_output/varying_res/02km/run_start_5km_guowei/'
ds_tracer_2_mean =  open_mdsdataset(dir_mean+'mean', grid_dir= dir_mean
                                    ,ignore_unknown_vars=True, geometry='cartesian', ref_date = "2100-1-1 0:0:0")

/home/dbalwada/miniconda/envs/dhruvenv/lib/python2.7/site-packages/xmitgcm/mds_store.py:696: UserWarning: Couldn't find available_diagnostics.log in /swot/SUM05/dbalwada/channel_model_output/varying_res/02km/run_start_5km_guowei/mean. Using default version.
  "in %s. Using default version." % data_dir)


In [5]:
# EKE 

grid = Grid(ds_tracer_2)
Ushift     = grid.interp(ds_tracer_2.U,axis='X')
Umeanshift = grid.interp(ds_tracer_2_mean.uVeltave.isel(time=-1),axis='X')

Vshift     = grid.interp(ds_tracer_2.V,axis='Y')
Vmeanshift = grid.interp(ds_tracer_2_mean.vVeltave.isel(time=-1),axis='Y')


Uprime = Ushift - Umeanshift
Vprime = Vshift - Vmeanshift

EKE = 0.5*(Uprime**2 + Vprime**2)
EKE.name = 'EKE'

# Vel gradients

dy = ds_tracer_2.dyC[0,0].values
dx = ds_tracer_2.dxC[0,0].values

mask_s = 1. #~(ds_tracer_2.hFacS==0) 
mask_w = 1. #~(ds_tracer_2.hFacW==0) 


dudx = grid.interp(grid.diff(Uprime, axis='X')/dx*mask_w, axis='X')
dvdx = grid.interp(grid.diff(Vprime, axis='X')/dx*mask_w, axis='X')

dudy = grid.interp(grid.diff(Uprime, axis='Y')/dy*mask_s, axis='Y')
dvdy = grid.interp(grid.diff(Vprime, axis='Y')/dy*mask_s, axis='Y')

vorticity = (dvdx - dudy)
vorticity.name = 'zeta'
strain = ((dudx-dvdy)**2 + (dudy+dvdx)**2)**0.5
strain.name = 'strain'
divergence = dudx + dvdy
divergence.name = 'div'

# modulus of horizontal buoyancy gradients 
g =9.81
rho_o = 1000.
alpha = 2e-4
dTdy = mask_s*grid.diff(ds_tracer_2['T'], axis='Y')/dy
dTdx = mask_w*grid.diff(ds_tracer_2['T'], axis='X')/dx

dbdy = g*alpha*grid.interp(dTdy, axis='Y')
dbdx = g*alpha*grid.interp(dTdx, axis='X')
        
gradb = (dbdx**2 + dbdy**2)**(0.5)
gradb.name = 'gradb'

In [ ]:
strain

In [ ]:
vorticity.isel(time=-1,YC=-3).plot(vmin=-0.0002)

In [ ]:
vorticity.isel(time=-1,YC=-2).plot(vmin=-0.0002)

In [ ]:
vorticity.isel(time=-1,YC=-200).plot()

In [ ]:
gradb.nbytes/1e9

In [ ]:
# PV
f_o  = -1.1e-4
beta = 1.4e-11 

f = f_o + beta*ds_tracer_2.YC

Z = ds_tracer_2.Z
Zmid = 0.5*(Z[0:-1]+Z[:])

dz = Z.diff(dim='Z')
dT = ds_tracer_2['T'].diff(dim='Z')
        
dbdz = dT/dz*g*alpha

In [ ]:
dT

In [ ]:
dbdzint = np.zeros_like(ds_tracer_2['T'])
dbdzint[:,0:39,:,:] =  dbdz.values[:,:,:,:]
dbdz = xr.DataArray(dbdzint, dims=ds_tracer_2['T'].dims, coords=ds_tracer_2['T'].coords)
        
N2 = dbdz

In [ ]:
N2.name = 'N2'
verPV = dbdz*(f+vorticity)
verPV.name = 'PV'

In [7]:
#ds_tracer_2.Wi = ds_tracer_2.W
ds_tracer_2.W.variable.dims = ['time','Z','YC','XC']

In [ ]:
# save the fields (EKE, N2, PV, zeta, strain, divergence, grad B, Ptracer)

movie_fields = xr.Dataset({'EKE': EKE, 'N2':N2, 'PV':verPV, 'zeta':vorticity, 
                          'strain':strain, 'divergence':divergence, 'gradb':gradb,
                           'ptracer':ds_tracer_2.PTRACER01,'W':ds_tracer_2.W, 
                          'buoyancy':ds_tracer_2['T']*alpha*g})

In [8]:
movie_fields = xr.Dataset({'EKE': EKE, 'zeta':vorticity, 
                          'strain':strain, 'divergence':divergence, 'gradb':gradb,
                           'ptracer':ds_tracer_2.PTRACER01,'W':ds_tracer_2.W, 
                          'buoyancy':ds_tracer_2['T']*alpha*g})

In [9]:
movie_fields

<xarray.Dataset>
Dimensions:     (XC: 800, YC: 800, Z: 40, time: 720)
Coordinates:
  * YC          (YC) float32 1250.0 3750.0 6250.0 8750.0 11250.0 13750.0 ...
  * XC          (XC) float32 1250.0 3750.0 6250.0 8750.0 11250.0 13750.0 ...
  * Z           (Z) float32 -5.0 -15.0 -25.0 -36.0 -49.0 -64.0 -81.5 -102.0 ...
  * time        (time) datetime64[ns] 2144-05-10 2144-05-10T12:00:00 ...
    Depth       (YC, XC) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    rA          (YC, XC) float32 6.25e+06 6.25e+06 6.25e+06 6.25e+06 ...
    PHrefC      (Z) float32 49.05 147.15 245.25 353.16 480.69 627.84 799.515 ...
    drF         (Z) float32 10.0 10.0 10.0 12.0 14.0 16.0 19.0 22.0 26.0 ...
    hFacC       (Z, YC, XC) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    iter        (time) int64 4665600 4665744 4665888 4666032 4666176 4666320 ...
Data variables:
    strain      (time, Z, YC, XC) float32 5.24351e-05 4.80016e-05 ...
    W           (time, Z, YC, XC) float32 -0.0 -0.0

In [ ]:
movie_fields.to_netcdf('/swot/SUM05/dbalwada/channel_model_output/varying_res/02km/movie_vars_1year_fromgyre.nc')



